In [3]:
#IMPORTING NECESSARY PACKAGES 
import numpy as np 

In [26]:

def binomial_tree_option(N,T,S0,sigma,r,comp, K, call = True, array_out = False, compounding = True):
    """ 
    Description
    Binomial tree option pricing 

    Args: 
    N (int) = periods, 
    T (int) = time to maturity, 
    S0 (int/float) = spot price, 
    sigma (float) = volatility, 
    r (float) = interest rate, 
    comp (int) = compounding periods per year, 
    K (int/float) = strike price, 
    call (bool) = True(else put)
    array_out = False just the price, else binomial tree, prices of the option in each period and the price as the output
    compounding (bool) = True(continuous), else (not continuous)

    Returns:
    float: call OR put option price

    """

    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u

    if compounding:
        p = (np.exp(r*dt)-d)/(u-d)
    else:
        p = (((1+r/comp)**dt*comp)-d)/(u-d)
    
    #Binomial Tree
    price_tree = np.zeros([N+1, N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
    #Option value 
    option = np.zeros([N+1, N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1), price_tree[:,N]-K)
    else:
        option[:,N] = np.maximum(np.zeros(N+1), K - price_tree[:,N])
    #Calculate option price at t = 0

    for i in np.arange(N-1, -1,-1):
        for j in np.arange(0,i+1):
            if compounding: 
                option[j,i] = np.exp(-r*dt)*(p*option[j,i+1]+(1-p)*option[j+1, i+1])
            else: 
                option[j,i] = (p*option[j,i+1]+(1-p)*option[j+1, i+1])/((1+r/comp)**dt*comp)
    # Return
    if array_out:
        return[option[0,0], price_tree, option]
    else: 
        return option[0,0]


In [27]:
binomial_tree_option(N=4,T = 1,S0 = 100,sigma = 0.1,r = 0.03,comp = 1, K = 110, call = True, array_out = False, compounding = True)

1.3380184214015374

In [50]:

def binomial_tree_call_put(N,T,S0,u, d, r,comp, K, call = True, array_out = False, compounding = True):
    """ 
    Description:
 
    Binomial tree option pricing with known up and down movements as percentage of spot price 

    Args: 
    N (int) = periods, 
    T (int) = time to maturity, 
    S0 (int/float) = spot price, 
    u (float) = up movement as percentage from the t-1 period, 
    d (float) = down movement as percentage from t-1 period, 
    sigma (float) = volatility, 
    r (float) = interest rate, 
    comp (int) = compounding periods per year, 
    K (int/float) = strike price, 
    call (bool) = True(else put)
    array_out = False just the price, else binomial tree, prices of the option in each period and the price as the output
    compounding (bool) = True(continuous), else (not continuous)

    Returns:
    float: call OR put option price

    """
    u = 1 + u 
    d = 1 - d

    dt = T/N

    if compounding:
        p = (np.exp(r*dt)-d)/(u-d)
    else:
        p = (((1+r/comp)**dt*comp)-d)/(u-d)
    
    #Binomial Tree
    price_tree = np.zeros([N+1, N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
    
    #Option value 
    option = np.zeros([N+1, N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1), price_tree[:,N]-K)
    else:
        option[:,N] = np.maximum(np.zeros(N+1), K - price_tree[:,N])
    
    #Calculate option price at t = 0
    for i in np.arange(N-1, -1,-1):
        for j in np.arange(0,i+1):
            if compounding: 
                option[j,i] = np.exp(-r*dt)*(p*option[j,i+1]+(1-p)*option[j+1, i+1])
            else: 
                option[j,i] = (p*option[j,i+1]+(1-p)*option[j+1, i+1])/((1+r/comp)**dt*comp)
    # Return
    if array_out:
        return[option[0,0], price_tree, option]
    else: 
        return option[0,0]


In [51]:
binomial_tree_call_put(N=1,T = 2,S0 = 100,u = 0.3, d = 0.2,r = 0.1, comp = 1, K = 103, call = True, array_out = False, compounding = False)

18.29752066115703

In [54]:

def binomial_tree_call_put_currency(N,T,S0,u, d, r, r_fc, comp, K, call = True, array_out = False, compounding = True):
    """ 
    Description:

    Binomial tree currency option pricing with known up and down movements as percentage of spot price 

    Args: 
    N (int) = periods, 
    T (int) = time to maturity, 
    S0 (int/float) = spot price, 
    u (float) = up movement as percentage of the t-1 period, 
    d (float) = down movement as percentage of t-1 period, 
    sigma (float) = volatility, 
    r (float) = interest rate domestic currency, 
    r_fc (float) = interest rate foreign currency
    comp (int) = compounding periods per year, 
    K (int/float) = strike price, 
    call (bool) = True(else put)
    array_out = False just the price, else binomial tree, prices of the option in each period and the price as the output
    compounding (bool) = True(continuous), else (not continuous)

    Returns:
    float: call OR put option price


    """
    u = 1 + u 
    d = 1 - d

    dt = T/N
    if compounding:
        p = (np.exp(r*dt)-d)/(u-d)
    else:
        p = (((1+r_fc/comp)**(dt*comp))-d)/(u-d)
    #Binomial Tree
    price_tree = np.zeros([N+1, N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
    #Option value 
    option = np.zeros([N+1, N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1), price_tree[:,N]-K)
    else:
        option[:,N] = np.maximum(np.zeros(N+1), K - price_tree[:,N])
    #Calculate option price at t = 0
    for i in np.arange(N-1, -1,-1):
        for j in np.arange(0,i+1):
            if compounding: 
                option[j,i] = np.exp(-r*dt)*(p*option[j,i+1]+(1-p)*option[j+1, i+1])
            else: 
                option[j,i] = (p*option[j,i+1]+(1-p)*option[j+1, i+1])/((1+r/comp)**dt*comp)
    # Return
    if array_out:
        return[option[0,0], price_tree, option]
    else: 

        return option[0,0]


In [55]:
binomial_tree_call_put_currency(N=1,T = 2,S0 = 100,u = 0.3, d = 0.2,r = 0.1, r_fc = 0.1, comp = 1, K = 100, call = True, array_out = False, compounding = False)

20.3305785123967